In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline

# Business Understanding
Questions: Related to business or real-world applications of how the data could be used to solve


Q1 : What are the most popular programming languages among developers?

Q2 : How do different programming languages stands in terms of job satisfaction?

Q3 : Base programming language to start professional career?

Data Understanding (Common)

We will use the data of StackOverflow's Developer Survey for year 2017. 
#download dataset URL : https://www.kaggle.com/stackoverflow/so-survey-2017

In [2]:
data = {}
data['2017'] = pd.read_csv('./data/survey_results_public.csv')
#TODO planning to explore 2018/2019 data also

In [3]:
# Explore Missing value in data
for year in ['2017']:
    df = data[year]       
    print(f'data shape:{df.shape}\n')
    no_nulls_cols = set(df.columns[df.isnull().mean() == 0])
    print(f'columns with no missing values:\n{no_nulls_cols}\n')
    most_missing_cols = set(df.columns[df.isnull().mean() > .5])
    print(f'columns with more than 50% missing values:\n{most_missing_cols}\n')
    most_missing_cols = set(df.columns[df.isnull().mean() > .75])
    print(f'columns with more than 75% missing values:\n{most_missing_cols}\n')

data shape:(51392, 154)

columns with no missing values:
{'University', 'EmploymentStatus', 'Professional', 'Country', 'Respondent', 'ProgramHobby', 'FormalEducation'}

columns with more than 50% missing values:
{'AssessJobProjects', 'AssessJobDiversity', 'ExCoderActive', 'ExCoderBalance', 'ResumePrompted', 'CousinEducation', 'ExCoderBelonged', 'AssessJobTech', 'AssessJobCommute', 'WantWorkDatabase', 'EducationImportant', 'AssessJobCompensation', 'AssessJobOffice', 'Overpaid', 'InfluenceTechStack', 'AssessJobExp', 'WantWorkFramework', 'AssessJobRemote', 'AssessJobRole', 'ExCoderReturn', 'NonDeveloperType', 'TimeAfterBootcamp', 'MetricAssess', 'ExCoder10Years', 'InfluenceCommunication', 'AssessJobFinances', 'InfluenceRecruitment', 'WebDeveloperType', 'LearnedHiring', 'ExCoderNotForMe', 'ExCoderWillNotCode', 'AssessJobIndustry', 'InfluenceWorkstation', 'InfluenceDatabase', 'InfluenceDeptTech', 'InfluenceConsultants', 'YearsCodedJobPast', 'AssessJobProfDevel', 'InfluenceCloud', 'Influence

In [4]:
# check top 5 data
data['2017'].head()

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A professional degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,113750.0,NaN
3,4,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A doctoral degree,White or of European descent,Agree,Agree,Somewhat agree,Strongly agree,NaN,NaN
4,5,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Solutions



# What are the most popular programming languages among developers?
Data Understanding

In [5]:
# columns indicating what languages are used by individual developers
prog_langs_col_dict = {
    '2017': 'HaveWorkedLanguage'    
}
prog_langs_col_dict

{'2017': 'HaveWorkedLanguage'}

In [6]:
# check the proportion of missing values in these columns
for year, col in prog_langs_col_dict.items():
    print(year, data[year][col].isnull().mean())

2017 0.2873404420921544


# Prepare Data

In [7]:
def extract_languages(langs_str, delimiter=';'):
    """
    INPUT:
        langs_str - the string you want to separate 
    OUTPUT:
        a list with separated values
    """
    return [val.strip() for val in langs_str.split(delimiter)]

In [8]:
def all_vals_set(df, col):
    """
    INPUT:
        df - the pandas dataframe you want to search
        colmn - the column you want to collect unique values 
    OUTPUT:
        values_set - a set of unique values in df[col]
    """
    values_set = set()
    mixed_lst = df[col].value_counts().index.tolist()
    for mixed_val in mixed_lst:
        mixed_val_set = set(extract_languages(mixed_val))
        values_set = values_set | mixed_val_set
    return values_set

In [9]:
# Rertieve programming languages in particular year
all_langs = {}

for year, col in prog_langs_col_dict.items():
    all_langs[year] = all_vals_set(data[year], col)

all_langs

{'2017': {'Assembly',
  'C',
  'C#',
  'C++',
  'Clojure',
  'CoffeeScript',
  'Common Lisp',
  'Dart',
  'Elixir',
  'Erlang',
  'F#',
  'Go',
  'Groovy',
  'Hack',
  'Haskell',
  'Java',
  'JavaScript',
  'Julia',
  'Lua',
  'Matlab',
  'Objective-C',
  'PHP',
  'Perl',
  'Python',
  'R',
  'Ruby',
  'Rust',
  'SQL',
  'Scala',
  'Smalltalk',
  'Swift',
  'TypeScript',
  'VB.NET',
  'VBA',
  'Visual Basic 6'}}

In [10]:
# Get common programming languages in data
common_langs = None

for year in prog_langs_col_dict.keys():
    if common_langs is None:
        common_langs = all_langs[year]
        continue
    common_langs = common_langs & possible_langs[year]

common_langs

{'Assembly',
 'C',
 'C#',
 'C++',
 'Clojure',
 'CoffeeScript',
 'Common Lisp',
 'Dart',
 'Elixir',
 'Erlang',
 'F#',
 'Go',
 'Groovy',
 'Hack',
 'Haskell',
 'Java',
 'JavaScript',
 'Julia',
 'Lua',
 'Matlab',
 'Objective-C',
 'PHP',
 'Perl',
 'Python',
 'R',
 'Ruby',
 'Rust',
 'SQL',
 'Scala',
 'Smalltalk',
 'Swift',
 'TypeScript',
 'VB.NET',
 'VBA',
 'Visual Basic 6'}

In [11]:
def total_count_lang(df, look_for):
    """
    INPUT:
        df - the pandas dataframe you want to search
        look_for - a list of strings you want to search for in each row of df['language']
    OUTPUT:
        new_df - a dataframe of each look_for with the count of how often it shows up
    """
    new_df = defaultdict(int)
    for lang in look_for:
        for idx in range(df.shape[0]):
            if lang in extract_languages(df['language'][idx]):
                new_df[lang] += int(df['count'][idx])
    new_df = pd.DataFrame(pd.Series(new_df)).reset_index()
    new_df.columns = ['language', 'count']
    new_df.sort_values('count', ascending=False, inplace=True)
    return new_df

In [12]:
def clean_lang(df, col, possible_vals):
    """
    INPUT: 
        df - a dataframe holding the worked language column
        col - the column you want to clean values
        possible_vals - a list of strings you want to search for in each row of df[col]
    OUTPUT:
        new_df - a dataframe with the proportions of how many individuals
    """
    new_df = df[col].value_counts().reset_index()
    new_df.rename(columns={'index': 'language', col: 'count'}, inplace=True)
    new_df = total_count_lang(new_df, possible_vals)
    new_df['count'] = new_df['count']/len(df.dropna(subset=[col], how='any'))
    new_df['rank'] = np.arange(1, len(new_df)+1)
    new_df.set_index('language', inplace=True)
    return new_df

In [13]:
# clean up language columns
langs = {}
for year, col in prog_langs_col_dict.items():
    langs[year] = clean_lang(data[year], col, all_langs[year])

# Visualize the Results

In [14]:
def plot_lang(df, title):
    """
    INPUT:
        df - a dataframe holding the worked language column
        title - the title string of your plot         
    OUTPUT:
        None
    """
    df[['count']].plot(figsize=(10,6), kind='bar', legend=None)
    plt.tick_params(left=False, bottom=False)
    plt.grid(True, axis='y')
    plt.title(title)
    plt.box(False)
    plt.show();

In [15]:
# plot the proportion of languages in each year
for year, col in worked_langs_col_dict.items():
    plot_lang(langs[year], f'Programming Languages {year}')

NameError: name 'worked_langs_col_dict' is not defined

In [ ]:
# get rankings of common languages in 2017 year
langs_ranks = pd.DataFrame(index=common_langs)
for year in worked_langs_col_dict.keys():
    langs_ranks = langs_ranks.merge(langs[year], left_index=True, right_index=True, how='inner').drop('count', axis=1)
langs_ranks.columns = worked_langs_col_dict.keys()
langs_ranks.transpose()

In [ ]:
# plot rankings of different languages in 2017 year
def plot_rankings(df, max_rank, title):
    """
    INPUT:
        df - a dataframe you want to plot
        max_rank - the maximum value of rank
        title - the title string of your plot         
    OUTPUT:
        None
    """
    df.sort_values(by='2017', ascending=True).plot(
        figsize=(10, 10),
        kind='bar', 
        xlim=(max_rank, 0),
        xticks=range(1,max_rank),
    )
    plt.tick_params(labelright=True, left=False, right=False, top=False, bottom=False)
    plt.legend(bbox_to_anchor=(1, 1))
    plt.title(title)
    plt.box(False)
    plt.show();

In [ ]:
plot_rankings(langs_ranks, 40, 'Programming Language Rankings 2017')

# Solution 2 : How do different programming languages stands in terms of job satisfaction?


In [ ]:
# columns indicating job satisfaction
job_sat_col_dict = {
    '2017': 'JobSatisfaction',    
}

In [ ]:
# check the proportion of missing values in these columns
for year, col in job_sat_col_dict.items():
    print(year, data[year][col].isnull().mean())

In [ ]:
def plot_value_counts(col, df, title):
    """
    INPUT: 
        col - the column you want to count values
        df - the dataframe holding the column
        title - the title of the plot
    OUTPUT:
        None
    """
    feat_vals = df[col].value_counts()
    (feat_vals/df.shape[0]).sort_index().plot(kind="bar");
    plt.title(title)
    plt.show();

In [ ]:
# check the proportion of each value in these columns
for year, col in job_sat_col_dict.items():
    plot_value_counts(col, data[year], f'Job Satisfaction {year}')

# Prepare Data


In [ ]:
# Because each year survey has a different job satisfaction level, we have to re-encode those fine scales into the same rough scale before comparison
# column names for re-encoded columns
job_sat_enc_col_dict = {
    '2017': 'JobSatisfaction_encoded'
}

In [ ]:
# create a dict for re-encoding
job_sat_enc_dict = {
    '2017': {
        0: -2,
        1: -1.6,
        2: -1.2,
        3: -0.8,
        4: -0.4,
        5: 0,
        6: 0.4,
        7: 0.8,
        8: 1.2,
        9: 1.6,
        10: 2,
    }
}

In [ ]:
def total_count_job_sat(df, look_for):
    """
    INPUT:
        df - the pandas dataframe you want to search
        look_for - a list of strings you want to search for in each row of df['language']
    OUTPUT:
        new_df - a dataframe of each look_for with the count and the mean
    """
    s = [defaultdict(int), defaultdict(int)]
    for lang in look_for:
        for idx in range(df.shape[0]):
            if lang in extract_languages(df['language'][idx]):
                s[0][lang] += int(df['count'][idx])
                s[1][lang] += int(df['job_sat_sum'][idx])

    new_df = pd.concat([pd.Series(s[0]), pd.Series(s[1])], axis=1)
    new_df.reset_index(inplace=True)
    new_df.columns = ['language', 'count', 'job_sat_sum']
    
    new_df['job_sat_mean'] = new_df['job_sat_sum'] / new_df['count']
    new_df.sort_values('job_sat_mean', ascending=False, inplace=True)
    new_df.set_index('language', inplace=True)
    
    return new_df    

In [ ]:
def clean_job_sat(df, lang_col, possible_langs, job_col, job_enc_col, job_enc_dict):
    """
    INPUT:
        df - the pandas dataframe you want to search
        lang_col - the column name with languages strings
        possible_langs - a list of languages you want to search for in each row of df[lang_col]
        job_col - the column name with job satisfaction values
        job_enc_col - the re-encoded column name for job_col
        job_enc_dict - the mapping dictionary for re-encoding
    OUTPUT:
        new_df - a dataframe for each possible_langs with the count and the mean of job satisfaction
    """
    # re-encode Job Satisfaction column values
    df[job_enc_col] = df[job_col].map(job_enc_dict, na_action='ignore')

    # group by languages
    new_df = pd.DataFrame(df[lang_col].value_counts())
    new_df['job_sat_sum'] = df.groupby([lang_col]).sum()[job_enc_col]
    new_df = new_df.reset_index()
    new_df.rename(columns={'index': 'language', lang_col: 'count'}, inplace=True)
    
    # count languages and calculate the mean of job satisfaction of each language
    new_df = total_count_job_sat(new_df, possible_langs)
    new_df['count'] = new_df['count']/len(df.dropna(subset=[lang_col], how='any'))
    
    return new_df

In [ ]:
# clean up job satisfaction columns
job_sat = {}
        
for year in job_sat_col_dict.keys():
    
    lang_col = worked_langs_col_dict[year]
    possible_langs_ = all_langs[year]
    
    job_col = job_sat_col_dict[year]
    job_enc_col = job_sat_enc_col_dict[year]
    job_enc_dict = job_sat_enc_dict[year]
    
    job_sat[year] = clean_job_sat(data[year], lang_col, possible_langs_, job_col, job_enc_col, job_enc_dict)

# Visualize the Results

In [ ]:
def plot_job_sat(df, title):
    """
    INPUT:
        df - the pandas dataframe you want to plot
        title - the title string of your plot
    OUTPUT:
        None
    """
    plt.figure(figsize=(10, 6))
    
    ax1 = df['count'].plot(kind='bar')
    ax1.set_ylabel('Language Proportion')
    plt.box(False)

    ax2 = df['job_sat_mean'].plot(kind='line', secondary_y=True, rot=90)
    ax2.set_ylabel('Job Satisfaction')
    plt.box(False)

    for ax in [ax1, ax2]:
        ax.tick_params(left=False, right=False, bottom=False)
        ax.set_xlabel('')
        ax.set_ylim(0,1)

    plt.grid(True, axis='y')
    plt.title(title)
    plt.show();

In [ ]:
# plot the mean of job satisfaction with the proportion of each language
for year in job_sat_col_dict.keys():
    plot_job_sat(job_sat[year], f'Languages and Job Satisfaction {year}')

In [ ]:
# create a dataframe of job satisfaction for each common language in all years
job_sat_all = pd.DataFrame(index=common_langs)
for year in job_sat_col_dict.keys():
    job_sat_all = job_sat_all.merge(job_sat[year][['job_sat_mean']], left_index=True, right_index=True, how='inner')
job_sat_all.columns = job_sat_col_dict.keys()
job_sat_all.transpose()

In [ ]:
# plot job satisfaction for each common language in all years
df_plot = job_sat_all.sort_values(by='2017', ascending=False)
ax = df_plot.plot(
        figsize=(10,6),
        kind='bar', 
        legend=True,
        ylim=(0.3, 0.8),
    )
ax.set_ylabel('Job Satisfaction')
plt.grid(True, axis='y')
plt.title('Languages and Job Satisfaction 2017')
plt.tick_params(left=False, bottom=False)
plt.box(False)
plt.show();

# Solution 3:  Base programming language to start professional career?
Data Understanding
With 2017, there are two columns related to years of coding. We need to fill missing values in one column with the other column at first.

In [ ]:
# fill missing values with a column
data['2017']['YearsCodedJob'].fillna(data['2017']['YearsCodedJobPast'], inplace=True)

In [ ]:
# columns indicating years of code
yr_code_col_dict = {
    '2017': 'YearsCodedJob',
}

In [ ]:
# check the proportion of missing values in these columns
for year, col in yr_code_col_dict.items():
    print(year, data[year][col].isnull().mean())

In [ ]:
# check the proportion of each value in these columns
for year, col in yr_code_col_dict.items():
    plot_value_counts(col, data[year], f'Years of Code {year}')

Prepare Data
Because each survey has different levels in these columns, we have to convert those levels into the same rough scale before comparison.

In [ ]:
# create a dict for re-encoding
yr_code_enc_dict = {
    '2017': {},    
}

possible_yr_code_values = [
    '0-2 years',
    '3-5 years',
    '6-8 years',
    '9-11 years',
    '12-14 years', 
    '15-17 years', 
    '18-20 years', 
    '>20 years',
]

# year 2017
# 0-2 years
yr_code_enc_dict['2017'].update({k:'0-2 years' for k in ['Less than a year', '1 to 2 years']})
# 3-5 ~ 18-20 years
for i in range(3, 20, 3):
    lst = [f'{j} to {j+1} years' for j in range(i-1, i+2)]
    yr_code_enc_dict['2017'].update({k:f'{i}-{i+2} years' for k in lst})
# >20 years
yr_code_enc_dict['2017'].update({'20 or more years':'>20 years'}) 

In [ ]:
def total_count_yr_code(df, look_for):
    """
    INPUT:
        df - the pandas dataframe you want to search
        look_for - a list of strings you want to search for in each row of df['language']
    OUTPUT:
        new_df - a dataframe of each look_for with the proportion
    """
    new_df = pd.Series(0, index=pd.MultiIndex.from_product(
            [look_for, possible_yr_code_values],names=('language', 'years_of_code')))    
    mix_langs = df.index.levels[0]
    for idx in range(len(mix_langs)):
        mix_langs_lst = extract_languages(mix_langs[idx])
        for lang in look_for:
            if lang in mix_langs_lst:
                new_df[lang] = new_df[lang].add(df.loc[mix_langs[idx]]['count'], fill_value=0)
    return new_df

In [ ]:
def clean_yr_code(df, lang_col, possible_langs_, cod_col, cod_enc_dict):
    """
    INPUT: 
        df - a dataframe holding the years of coding column
        lang_col - the column name with languages strings
        possible_langs - a list of languages you want to search for in each row of df[lang_col]
        cod_col - the column name with years of coding values
        cod_enc_dict - the mapping dictionary for re-encoding
    OUTPUT:
        new_df - a dataframe for each possible_langs of years of coding
    """
    new_df = df[[lang_col, cod_col]].copy()
    
    # re-encode
    new_df[cod_col].replace(cod_enc_dict, inplace=True)
    
    # count
    new_df = new_df.groupby([lang_col, cod_col]).size().reset_index()
    new_df.columns = ['language', 'years_of_code', 'count']
    new_df.set_index(['language', 'years_of_code'], inplace=True)
    new_df = total_count_yr_code(new_df, possible_langs_)
    return new_df

In [ ]:
# clean up years of code columns
yr_code = {}

for year in yr_code_col_dict.keys():
    
    lang_col = worked_langs_col_dict[year]
    possible_langs_ = all_langs[year]
    
    cod_col = yr_code_col_dict[year]
    cod_enc_dict = yr_code_enc_dict[year]
    
    yr_code[year] = clean_yr_code(data[year], lang_col, possible_langs_, cod_col, cod_enc_dict)

# Visualize the Results

In [ ]:
def plot_yr_code(df, title, only_new_comer=False):
    """
    INPUT:
        df - the pandas dataframe you want to plot
        title - the title string of your plot
        only_new_comer - only plot 0-5 years data
    OUTPUT:
        None
    """
    # use proportion    
    df = df.div(df.sum(level=0), axis='index', level=0)
    # reindex() is for sorting years of code labels
    df = df.unstack(level=1).reindex(possible_yr_code_values, axis=1)
    if only_new_comer:
        df['0-5 years'] = df['0-2 years'] + df['3-5 years']
        df = df[['0-5 years']].sort_values(by='0-5 years', ascending=False)
        df['rank'] = np.arange(1, len(df)+1)
        df[['0-5 years']].plot.bar(figsize=(10,8))
        plt.yticks(np.arange(0.05, 0.6, 0.05))
        plt.xlabel('')
        plt.legend()
    else:
        df.plot.barh(figsize=(10,15), stacked=True, colormap=plt.cm.summer_r)
        plt.legend(bbox_to_anchor=(1.05, 1))
        plt.tick_params(labeltop=True, left=False, right=False, top=True, bottom=True)
        plt.xlim(0,1)
        plt.ylabel('')
        plt.gca().invert_yaxis()
    plt.title(title, pad=40)
    plt.show();
    return df

In [ ]:
# plot the proportion of years of code of each language in each year
for year in yr_code_col_dict.keys():
    plot_yr_code(yr_code[year], f'Languages and Years of Code {year}')

In [ ]:
# plot the proportion of new-comers of each language in each year
# and get rankings of common languages in all years
yr_code_ranks = pd.DataFrame(index=common_langs)

for year in yr_code_col_dict.keys():
    df = plot_yr_code(yr_code[year], f'Languages and Years of Code {year}', only_new_comer=True)
    yr_code_ranks = yr_code_ranks.merge(df, left_index=True, right_index=True, how='inner').drop('0-5 years', axis=1)

yr_code_ranks.columns = yr_code_col_dict.keys()

In [ ]:
yr_code_ranks.transpose()

In [ ]:
# plot rankings of new-comers for each common language in all years
plot_rankings(yr_code_ranks, 37, 'Languages and New-comers 2017')